In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy
import re
import string
from collections import Counter

In [83]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [173]:
import nltk

In [84]:
imdb = pd.read_csv('..\..\imdb_labelled.txt', sep='\t|', engine='python', header=None)
imdb.columns = ['desc', 'score']

C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2227: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
C:\Users\Ryan\Anaconda3\lib\site-packages\pandas\io\parsers.py:2229: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


In [85]:
with open('..\..\imdb_labelled.txt') as imdb_file: # almost always use the 'with' statement 
    text = imdb_file.readlines()    # when opening; dont have to use .close()
    print("This file is {} lines long\n".format(len(text)))
    for line in text:
        print(line)

This file is 1000 lines long

A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  	0

Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  	0

Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  	0

Very little music or anything to speak of.  	0

The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  	1

The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  	0

Wasted two hours.  	0

Saw the movie today and thought it was a good effort, good messages for kids.  	1

A bit predictable.  	0

Loved the casting of Jimmy Buffet as the science teacher.  	1

And those baby owls were adorable.  	1

The movie showed a lot of Florida at it's best, made it look very appeali

In [176]:
text = nltk.word_tokenize(imdb.desc[0])

In [177]:
text

['A',
 'very',
 ',',
 'very',
 ',',
 'very',
 'slow-moving',
 ',',
 'aimless',
 'movie',
 'about',
 'a',
 'distressed',
 ',',
 'drifting',
 'young',
 'man',
 '.']

In [178]:
nltk.pos_tag(text)

[('A', 'DT'),
 ('very', 'RB'),
 (',', ','),
 ('very', 'RB'),
 (',', ','),
 ('very', 'RB'),
 ('slow-moving', 'JJ'),
 (',', ','),
 ('aimless', 'JJ'),
 ('movie', 'NN'),
 ('about', 'IN'),
 ('a', 'DT'),
 ('distressed', 'JJ'),
 (',', ','),
 ('drifting', 'VBG'),
 ('young', 'JJ'),
 ('man', 'NN'),
 ('.', '.')]

In [208]:
comments = imdb.desc.tolist()

In [237]:
joined_comments = ''.join(comments)

In [267]:
joined_comments

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  Very little music or anything to speak of.  The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  The rest of the movie lacks art, charm, meaning... If it\'s about emptiness, it works I guess because it\'s empty.  Wasted two hours.  Saw the movie today and thought it was a good effort, good messages for kids.  A bit predictable.  Loved the casting of Jimmy Buffet as the science teacher.  And those baby owls were adorable.  The movie showed a lot of Florida at it\'s best, made it look very appealing.  The Songs Were The Best And The Muppets Were So Hilarious.  It Wa

In [253]:
token_words = nltk.word_tokenize(joined_comments)

In [275]:
nltk.pos_tag(token_words, tagset='universal')

[('A', 'DET'),
 ('very', 'ADV'),
 (',', '.'),
 ('very', 'ADV'),
 (',', '.'),
 ('very', 'ADV'),
 ('slow-moving', 'ADJ'),
 (',', '.'),
 ('aimless', 'ADJ'),
 ('movie', 'NOUN'),
 ('about', 'ADP'),
 ('a', 'DET'),
 ('distressed', 'ADJ'),
 (',', '.'),
 ('drifting', 'VERB'),
 ('young', 'ADJ'),
 ('man', 'NOUN'),
 ('.', '.'),
 ('Not', 'ADV'),
 ('sure', 'ADJ'),
 ('who', 'PRON'),
 ('was', 'VERB'),
 ('more', 'ADJ'),
 ('lost', 'ADJ'),
 ('-', '.'),
 ('the', 'DET'),
 ('flat', 'ADJ'),
 ('characters', 'NOUN'),
 ('or', 'CONJ'),
 ('the', 'DET'),
 ('audience', 'NOUN'),
 (',', '.'),
 ('nearly', 'ADV'),
 ('half', 'NOUN'),
 ('of', 'ADP'),
 ('whom', 'PRON'),
 ('walked', 'VERB'),
 ('out', 'PRT'),
 ('.', '.'),
 ('Attempting', 'VERB'),
 ('artiness', 'NOUN'),
 ('with', 'ADP'),
 ('black', 'ADJ'),
 ('&', 'CONJ'),
 ('white', 'ADJ'),
 ('and', 'CONJ'),
 ('clever', 'ADJ'),
 ('camera', 'NOUN'),
 ('angles', 'NOUN'),
 (',', '.'),
 ('the', 'DET'),
 ('movie', 'NOUN'),
 ('disappointed', 'ADJ'),
 ('-', '.'),
 ('became', 'VERB'

In [277]:
part_of_speech = nltk.pos_tag(token_words, tagset='universal')

In [370]:
imdb['desc2'] = imdb['desc'].apply(lambda x: ' '.join(nltk.word_tokenize(x)))

In [428]:
imdb['pos'] = imdb['desc2'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x), tagset='universal'))

In [335]:
# 'ADJ' and 'NUM'
imdb['adj'] = imdb['pos'].apply(lambda x: ' '.join([word[0] for word in x if word[1]=='ADJ' or word[1]=='NUM']))

In [319]:
imdb['adv'] = imdb['pos'].apply(lambda x: ' '.join([word[0] for word in x if word[1]=='ADV']))

In [338]:
imdb['verb'] = imdb['pos'].apply(lambda x: ' '.join([word[0] for word in x if word[1]=='VERB']))

In [442]:
imdb['noun'] = imdb['pos'].apply(lambda x: ' '.join([word[0] for word in x if word[1]=='NOUN']))

In [348]:
imdb['adj_adv_verb'] = imdb[['adj', 'adv', 'verb']].apply(lambda x: ' '.join(x), axis=1)

In [355]:
imdb['adj_verb'] = imdb[['adj', 'verb']].apply(lambda x: ' '.join(x), axis=1)

In [441]:
imdb['adj_adv'] = imdb[['adj', 'adv']].apply(lambda x: ' '.join(x), axis=1)

In [443]:
imdb['adj_noun'] = imdb[['adj', 'noun']].apply(lambda x: ' '.join(x), axis=1)

In [478]:
imdb['adj_noun_verb'] = imdb[['adj', 'noun', 'verb']].apply(lambda x: ' '.join(x), axis=1)

In [479]:
imdb['adj_adv_noun_verb'] = imdb[['adj', 'adv', 'noun', 'verb']].apply(lambda x: ' '.join(x), axis=1)

In [430]:
imdb['pos_only'] = imdb['pos'].apply(lambda x: ' '.join([tag[1] for tag in x]))

In [452]:
imdb['pos_str'] = imdb['pos'].apply(lambda x: str(x).strip('[]'))

In [453]:
imdb

,desc,score,categ,good,desc2,pos,adj,adv,number,verb,adj_adv_verb,adj_verb,pos_only,pos_str,adj_adv,noun,adj_noun
0,"A very, very, very slow-moving, aimless movie ...",0,neg,0,"A very , very , very slow-moving , aimless mov...","[(A, DET), (very, ADV), (,, .), (very, ADV), (...",slow-moving aimless distressed young,very very very,,drifting,slow-moving aimless distressed young very very...,slow-moving aimless distressed young drifting,DET ADV . ADV . ADV ADJ . ADJ NOUN ADP DET ADJ...,"('A', 'DET'), ('very', 'ADV'), (',', '.'), ('v...",slow-moving aimless distressed young very very...,movie man,slow-moving aimless distressed young movie man
1,Not sure who was more lost - the flat characte...,0,neg,0,Not sure who was more lost - the flat characte...,"[(Not, ADV), (sure, ADJ), (who, PRON), (was, V...",sure more lost flat,Not nearly,,was walked,sure more lost flat Not nearly was walked,sure more lost flat was walked,ADV ADJ PRON VERB ADJ ADJ . DET ADJ NOUN CONJ ...,"('Not', 'ADV'), ('sure', 'ADJ'), ('who', 'PRON...",sure more lost flat Not nearly,characters audience half,sure more lost flat characters audience half
2,Attempting artiness with black & white and cle...,0,neg,0,Attempting artiness with black & white and cle...,"[(Attempting, VERB), (artiness, NOUN), (with, ...",black white clever disappointed ridiculous poo...,even more almost,,Attempting became was,black white clever disappointed ridiculous poo...,black white clever disappointed ridiculous poo...,VERB NOUN ADP ADJ CONJ ADJ CONJ ADJ NOUN NOUN ...,"('Attempting', 'VERB'), ('artiness', 'NOUN'), ...",black white clever disappointed ridiculous poo...,artiness camera angles movie acting plot lines,black white clever disappointed ridiculous poo...
3,Very little music or anything to speak of.,0,neg,0,Very little music or anything to speak of .,"[(Very, ADV), (little, ADJ), (music, NOUN), (o...",little,Very,,speak,little Very speak,little speak,ADV ADJ NOUN CONJ NOUN PRT VERB ADP .,"('Very', 'ADV'), ('little', 'ADJ'), ('music', ...",little Very,music anything,little music anything
4,The best scene in the movie was when Gerardo i...,1,pos,0,The best scene in the movie was when Gerardo i...,"[(The, DET), (best, ADJ), (scene, NOUN), (in, ...",best,when,,was is trying find keeps running,best when was is trying find keeps running,best was is trying find keeps running,DET ADJ NOUN ADP DET NOUN VERB ADV NOUN VERB V...,"('The', 'DET'), ('best', 'ADJ'), ('scene', 'NO...",best when,scene movie Gerardo song head,best scene movie Gerardo song head
5,"The rest of the movie lacks art, charm, meanin...",0,neg,0,"The rest of the movie lacks art , charm , mean...","[(The, DET), (rest, NOUN), (of, ADP), (the, DE...",empty,art,,lacks 's works 's,empty art lacks 's works 's,empty lacks 's works 's,DET NOUN ADP DET NOUN VERB ADV . NOUN . NOUN ....,"('The', 'DET'), ('rest', 'NOUN'), ('of', 'ADP'...",empty art,rest movie charm meaning emptiness guess,empty rest movie charm meaning emptiness guess
6,Wasted two hours.,0,neg,0,Wasted two hours .,"[(Wasted, VERB), (two, NUM), (hours, NOUN), (....",two,,two,Wasted,two Wasted,two Wasted,VERB NUM NOUN .,"('Wasted', 'VERB'), ('two', 'NUM'), ('hours', ...",two,hours,two hours
7,Saw the movie today and thought it was a good ...,1,pos,1,Saw the movie today and thought it was a good ...,"[(Saw, NOUN), (the, DET), (movie, NOUN), (toda...",good good,,,thought was,good good thought was,good good thought was,NOUN DET NOUN NOUN CONJ VERB PRON VERB DET ADJ...,"('Saw', 'NOUN'), ('the', 'DET'), ('movie', 'NO...",good good,Saw movie today effort messages kids,good good Saw movie today effort messages kids
8,A bit predictable.,0,neg,0,A bit predictable .,"[(A, DET), (bit, NOUN), (predictable, ADJ), (....",predictable,,,,predictable,predictable,DET NOUN ADJ .,"('A', 'DET'), ('bit', 'NOUN'), ('predictable',...",predictable,bit,predictable bit
9,Loved the casting of Jimmy Buffet as the scien...,1,pos,0,Loved the casting of Jimmy Buffet as the scien...,"[(Loved, VERB), (the,

In [306]:
[item for item in part_of_speech if item[1] == 'NUM']

[('two', 'NUM'),
 ('Two', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('2', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('9', 'NUM'),
 ('10', 'NUM'),
 ('One', 'NUM'),
 ('one', 'NUM'),
 ('2', 'NUM'),
 ('one', 'NUM'),
 ('10', 'NUM'),
 ('one', 'NUM'),
 ('10', 'NUM'),
 ('one', 'NUM'),
 ('10/10', 'NUM'),
 ('1', 'NUM'),
 ('thousand', 'NUM'),
 ('five', 'NUM'),
 ('7.50', 'NUM'),
 ('one', 'NUM'),
 ('two', 'NUM'),
 ('one', 'NUM'),
 ('80s', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('One', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('1', 'NUM'),
 ('10', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('three', 'NUM'),
 ('Three', 'NUM'),
 ('three', 'NUM'),
 ('three', 'NUM'),
 ('10', 'NUM'),
 ('10', 'NUM'),
 ('one', 'NUM'),
 ('one', 'NUM'),
 ('25', 'NUM'),
 ('One', 'NUM'),
 ('5', 'NUM'),
 ('10', 'NUM'),
 ('one', 'NUM'),
 ('1', 'NUM'),
 ('8', 'NUM'),
 ('10', 'NUM'),
 ('10+', 'NUM'),
 ('four', 'NUM'),
 ('one', 'NUM'),
 ('90', 'NUM'),
 ('70000', 'NUM'),
 ('90', 'NUM'),
 ('90', 'NUM'),
 ('on

In [283]:
[item for item in part_of_speech if item[0] == '10/10']

[('10/10', 'NUM'), ('10/10', 'NUM'), ('10/10', 'NUM')]

In [286]:
adj = [word for word in part_of_speech if word[1] == 'ADJ']

In [293]:
adv = [word for word in part_of_speech if word[1] == 'ADV']

In [268]:
text = nltk.Text(word.lower() for word in joined_comments.split())

In [269]:
text

<Text: a very, very, very slow-moving, aimless movie about...>

In [672]:
text.similar('good')

wonderful movie real great better bad a very the half film other their
but fun mediocre family told lesser really


#### only use of SGDClassifier is for 'desc'
- results inconclusive; probably helps to cross-validate

In [617]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                         alpha=1e-3, random_state=42,
                                         max_iter=5, tol=None)),
])

In [309]:
text_mnb = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())
                    ])

##### using 'desc'

In [661]:
X_train, X_test, y_train, y_test = train_test_split(imdb.desc, imdb.score)

In [662]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [663]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [664]:
predicted1 = text_clf.predict(X_test)

In [665]:
predicted = text_mnb.predict(X_test)

In [666]:
np.mean(predicted1 == y_test)

0.808

In [667]:
np.mean(predicted == y_test)

0.8

In [668]:
print(metrics.classification_report(y_test, predicted1, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.80      0.78      0.79       115
        pos       0.82      0.83      0.82       135

avg / total       0.81      0.81      0.81       250



In [669]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.74      0.87      0.80       115
        pos       0.87      0.74      0.80       135

avg / total       0.81      0.80      0.80       250



In [670]:
metrics.confusion_matrix(y_test, predicted1)

array([[ 90,  25],
       [ 23, 112]], dtype=int64)

In [671]:
metrics.confusion_matrix(y_test, predicted)

array([[100,  15],
       [ 35, 100]], dtype=int64)

##### using 'adj'

In [609]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [610]:
predicted = text_mnb.predict(X_test)

In [611]:
np.mean(predicted == y_test)

0.7

In [612]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.63      0.85      0.73       117
        pos       0.81      0.57      0.67       133

avg / total       0.73      0.70      0.70       250



In [613]:
metrics.confusion_matrix(y_test, predicted)

array([[99, 18],
       [57, 76]], dtype=int64)

##### using 'adv'

In [326]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adv, imdb.score)

In [327]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [328]:
predicted = text_mnb.predict(X_test)

In [329]:
np.mean(predicted == y_test)

0.556

In [330]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.64      0.33      0.43       129
        pos       0.53      0.80      0.64       121

avg / total       0.58      0.56      0.53       250



In [331]:
metrics.confusion_matrix(y_test, predicted)

array([[42, 87],
       [24, 97]], dtype=int64)

##### using 'verb'

In [340]:
X_train, X_test, y_train, y_test = train_test_split(imdb.verb, imdb.score)

In [341]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [342]:
predicted = text_mnb.predict(X_test)

In [343]:
np.mean(predicted == y_test)

0.592

In [344]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.55      0.76      0.64       118
        pos       0.67      0.44      0.53       132

avg / total       0.62      0.59      0.58       250



In [345]:
metrics.confusion_matrix(y_test, predicted)

array([[90, 28],
       [74, 58]], dtype=int64)

##### using 'adj', 'adv', and 'verb'

In [349]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_adv_verb, imdb.score)

In [350]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [351]:
predicted = text_mnb.predict(X_test)

In [352]:
np.mean(predicted == y_test)

0.756

In [353]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.76      0.76      0.76       127
        pos       0.75      0.76      0.75       123

avg / total       0.76      0.76      0.76       250



In [354]:
metrics.confusion_matrix(y_test, predicted)

array([[96, 31],
       [30, 93]], dtype=int64)

##### 'adj' and 'verb'

In [363]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_verb, imdb.score)

In [357]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [358]:
predicted = text_mnb.predict(X_test)

In [359]:
np.mean(predicted == y_test)

0.752

In [360]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.72      0.79      0.75       120
        pos       0.79      0.72      0.75       130

avg / total       0.76      0.75      0.75       250



In [361]:
metrics.confusion_matrix(y_test, predicted)

array([[95, 25],
       [37, 93]], dtype=int64)

##### using tokenized 'desc'
- does this actually produce different results?

In [414]:
X_train, X_test, y_train, y_test = train_test_split(imdb.desc2, imdb.score)

In [415]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [416]:
predicted = text_mnb.predict(X_test)

In [417]:
np.mean(predicted == y_test)

0.844

In [418]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.85      0.84      0.85       129
        pos       0.84      0.84      0.84       121

avg / total       0.84      0.84      0.84       250



In [419]:
metrics.confusion_matrix(y_test, predicted)

array([[109,  20],
       [ 19, 102]], dtype=int64)

##### using only pos

In [432]:
X_train, X_test, y_train, y_test = train_test_split(imdb.pos_only, imdb.score)

In [433]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [434]:
predicted = text_mnb.predict(X_test)

In [435]:
np.mean(predicted == y_test)

0.556

In [436]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.55      0.57      0.56       124
        pos       0.56      0.54      0.55       126

avg / total       0.56      0.56      0.56       250



In [437]:
metrics.confusion_matrix(y_test, predicted)

array([[71, 53],
       [58, 68]], dtype=int64)

##### using word with pos

In [550]:
X_train, X_test, y_train, y_test = train_test_split(imdb.pos_str, imdb.score)

In [551]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [552]:
predicted = text_mnb.predict(X_test)

In [553]:
np.mean(predicted == y_test)

0.78

In [554]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.70      0.92      0.79       114
        pos       0.91      0.66      0.77       136

avg / total       0.81      0.78      0.78       250



In [555]:
metrics.confusion_matrix(y_test, predicted)

array([[105,   9],
       [ 46,  90]], dtype=int64)

##### just 'noun'

In [556]:
X_train, X_test, y_train, y_test = train_test_split(imdb.noun, imdb.score)

In [557]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [558]:
predicted = text_mnb.predict(X_test)

In [559]:
np.mean(predicted == y_test)

0.644

In [560]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.70      0.56      0.62       131
        pos       0.60      0.74      0.66       119

avg / total       0.65      0.64      0.64       250



In [561]:
metrics.confusion_matrix(y_test, predicted)

array([[73, 58],
       [31, 88]], dtype=int64)

#####  using 'adj' and 'adv'

In [562]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_adv, imdb.score)

In [563]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [564]:
predicted = text_mnb.predict(X_test)

In [565]:
np.mean(predicted == y_test)

0.728

In [566]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.82      0.62      0.70       131
        pos       0.67      0.85      0.75       119

avg / total       0.75      0.73      0.73       250



In [567]:
metrics.confusion_matrix(y_test, predicted)

array([[ 81,  50],
       [ 18, 101]], dtype=int64)

#####  using 'adj' and 'noun'

In [568]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_noun, imdb.score)

In [569]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [570]:
predicted = text_mnb.predict(X_test)

In [571]:
np.mean(predicted == y_test)

0.756

In [572]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.81      0.70      0.75       130
        pos       0.72      0.82      0.76       120

avg / total       0.76      0.76      0.76       250



In [573]:
metrics.confusion_matrix(y_test, predicted)

array([[91, 39],
       [22, 98]], dtype=int64)

##### using 'adj', 'noun', 'verb'

In [574]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_noun_verb, imdb.score)

In [575]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [576]:
predicted = text_mnb.predict(X_test)

In [577]:
np.mean(predicted == y_test)

0.812

In [578]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.85      0.80      0.82       137
        pos       0.77      0.83      0.80       113

avg / total       0.81      0.81      0.81       250



In [579]:
metrics.confusion_matrix(y_test, predicted)

array([[109,  28],
       [ 19,  94]], dtype=int64)

##### using 'adj', 'adv', 'noun', and 'verb'

In [580]:
X_train, X_test, y_train, y_test = train_test_split(imdb.adj_adv_noun_verb, imdb.score)

In [581]:
text_mnb.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [582]:
predicted = text_mnb.predict(X_test)

In [583]:
np.mean(predicted == y_test)

0.784

In [584]:
print(metrics.classification_report(y_test, predicted, target_names=['neg', 'pos']))

             precision    recall  f1-score   support

        neg       0.82      0.75      0.79       132
        pos       0.75      0.82      0.78       118

avg / total       0.79      0.78      0.78       250



In [585]:
metrics.confusion_matrix(y_test, predicted)

array([[99, 33],
       [21, 97]], dtype=int64)

In [103]:
count_vect = CountVectorizer()

In [104]:
X_train_counts = count_vect.fit_transform(X_train)

In [105]:
count_vect.vocabulary_

{'the': 2196,
 'sets': 1941,
 'especially': 720,
 'designed': 570,
 'to': 2245,
 'work': 2504,
 'with': 2489,
 'camera': 315,
 'are': 137,
 'amazing': 98,
 'stylized': 2103,
 'beautiful': 207,
 'and': 108,
 'effective': 669,
 'beware': 237,
 'this': 2214,
 'is': 1171,
 'not': 1500,
 'trashy': 2282,
 'cult': 514,
 'movie': 1444,
 'trash': 2281,
 'period': 1604,
 'an': 106,
 'excellent': 741,
 'film': 826,
 'had': 988,
 'walk': 2417,
 'out': 1554,
 'of': 1522,
 'theatre': 2198,
 'for': 863,
 'few': 821,
 'minutes': 1414,
 'just': 1213,
 'get': 932,
 'some': 2015,
 'relief': 1798,
 'it': 1177,
 'presents': 1683,
 'idyllic': 1095,
 'yet': 2531,
 'serious': 1937,
 'portrayal': 1661,
 'ups': 2367,
 'downs': 626,
 'characters': 359,
 'lives': 1306,
 'manna': 1361,
 'from': 893,
 'heaven': 1024,
 'terrific': 2191,
 'that': 2195,
 'both': 273,
 'predictable': 1676,
 'unpredictable': 2357,
 'at': 158,
 'same': 1870,
 'time': 2238,
 'let': 1279,
 'my': 1459,
 'girlfriend': 940,
 'talk': 2166,
 'm

In [95]:
from sklearn.datasets import fetch_20newsgroups

In [106]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

In [107]:
twenty_train = fetch_20newsgroups(subset='train',
...     categories=categories, shuffle=True, random_state=42)

In [108]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [109]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [110]:
twenty_train.target_names[twenty_train.target[0]]

'comp.graphics'

In [111]:
# from sklearn.feature_extraction.text import CountVectorizer

In [112]:
count_vect = CountVectorizer()

In [113]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [615]:
X_train_counts

<2257x35788 sparse matrix of type '<class 'numpy.int64'>'
	with 365886 stored elements in Compressed Sparse Row format>

In [115]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [116]:
count_vect.vocabulary_

{'from': 14887,
 'sd345': 29022,
 'city': 8696,
 'ac': 4017,
 'uk': 33256,
 'michael': 21661,
 'collier': 9031,
 'subject': 31077,
 'converting': 9805,
 'images': 17366,
 'to': 32493,
 'hp': 16916,
 'laserjet': 19780,
 'iii': 17302,
 'nntp': 23122,
 'posting': 25663,
 'host': 16881,
 'hampton': 16082,
 'organization': 23915,
 'the': 32142,
 'university': 33597,
 'lines': 20253,
 '14': 587,
 'does': 12051,
 'anyone': 5201,
 'know': 19458,
 'of': 23610,
 'good': 15576,
 'way': 34755,
 'standard': 30623,
 'pc': 24651,
 'application': 5285,
 'pd': 24677,
 'utility': 33915,
 'convert': 9801,
 'tif': 32391,
 'img': 17389,
 'tga': 32116,
 'files': 14281,
 'into': 18268,
 'format': 14676,
 'we': 34775,
 'would': 35312,
 'also': 4808,
 'like': 20198,
 'do': 12014,
 'same': 28619,
 'hpgl': 16927,
 'plotter': 25361,
 'please': 25337,
 'email': 12833,
 'any': 5195,
 'response': 27836,
 'is': 18474,
 'this': 32270,
 'correct': 9932,
 'group': 15837,
 'thanks': 32135,
 'in': 17556,
 'advance': 4378,

In [117]:
# from sklearn.feature_extraction.text import TfidfTransformer

In [118]:
# tf
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [119]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [120]:
# tf-idf; Term Frequency times Inverse Document Frequency
tfidf_transformer = TfidfTransformer()

In [121]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [122]:
# from sklearn.naive_bayes import MultinomialNB

In [123]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [124]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']

In [125]:
X_new_counts = count_vect.transform(docs_new)

In [126]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [127]:
predicted = clf.predict(X_new_tfidf)

In [128]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' %(doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [129]:
# make vectorizer => transformer => classifier easier to work with...
# Pipeline class
# from sklearn.pipeline import Pipeline

In [130]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
])

In [131]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [132]:
twenty_test = fetch_20newsgroups(subset='test',
                          categories=categories, shuffle=True, random_state=42)

In [133]:
docs_test = twenty_test.data

In [134]:
predicted = text_clf.predict(docs_test)

In [135]:
np.mean(predicted == twenty_test.target)

0.8348868175765646

In [136]:
# support vector machine (SVM), regarded as one of best text clf algorithms 
# (but slow)
from sklearn.linear_model import SGDClassifier

In [137]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                         alpha=1e-3, random_state=42,
                                         max_iter=5, tol=None)),
])

In [138]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [139]:
predicted = text_clf.predict(docs_test)

In [140]:
np.mean(predicted==twenty_test.target)

0.9127829560585885

In [141]:
# from sklearn import metrics

In [142]:
print(metrics.classification_report(twenty_test.target, predicted,
                                   target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [143]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

In [144]:
a = imdb['desc']

In [145]:
line_length = []
for l in a:
    line_length.append(len(re.split(', | |-|:|\\n|\\t  ', l)))


In [146]:
l = []
for i, row in imdb.iterrows():
    if 'good' in row['desc']:
        l.append(1)
    else:
        l.append(0)
        

In [147]:
imdb['good'] = l

In [148]:
for i, row in imdb.iterrows():
    print (row)

desc     A very, very, very slow-moving, aimless movie ...
score                                                    0
categ                                                  neg
good                                                     0
Name: 0, dtype: object
desc     Not sure who was more lost - the flat characte...
score                                                    0
categ                                                  neg
good                                                     0
Name: 1, dtype: object
desc     Attempting artiness with black & white and cle...
score                                                    0
categ                                                  neg
good                                                     0
Name: 2, dtype: object
desc     Very little music or anything to speak of.  
score                                               0
categ                                             neg
good                                                0
Name: 3

desc     Barney is definitely super DE duper.  
score                                         1
categ                                       pos
good                                          0
Name: 343, dtype: object
desc     is pretty funny.  
score                     1
categ                   pos
good                      0
Name: 344, dtype: object
desc     Babie Bop is very cute.  
score                            1
categ                          pos
good                             0
Name: 345, dtype: object
desc     The kids are very cool too.  
score                                1
categ                              pos
good                                 0
Name: 346, dtype: object
desc     " But "Storm Trooper" is not even bad enough t...
score                                                    0
categ                                                  neg
good                                                     0
Name: 347, dtype: object
desc     It's just lame.  
score       

Name: 653, dtype: object
desc     There still are good actors around!  
score                                        1
categ                                      pos
good                                         1
Name: 654, dtype: object
desc     Also great directing and photography.  
score                                          1
categ                                        pos
good                                           0
Name: 655, dtype: object
desc     Very true to Shakespear, and a 'must' for all ...
score                                                    1
categ                                                  pos
good                                                     0
Name: 656, dtype: object
desc     Macbeth (Jason Connery) moved me to tears with...
score                                                    1
categ                                                  pos
good                                                     0
Name: 657, dtype: object
desc     Helen Baxe

desc     I have recommended it to friends.  
score                                      1
categ                                    pos
good                                       0
Name: 959, dtype: object
desc     I was particularly pleased with the acting abi...
score                                                    1
categ                                                  pos
good                                                     0
Name: 960, dtype: object
desc     Both actors truly understand and become their ...
score                                                    1
categ                                                  pos
good                                                     0
Name: 961, dtype: object
desc     Their on-screen chemistry, critical to the ent...
score                                                    1
categ                                                  pos
good                                                     0
Name: 962, dtype: object
desc     The

In [149]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [150]:

# words = re.split(r'\W+', a[0])
words = a[788].split()
table = str.maketrans('', '', string.punctuation)
stripped = [w.lower().translate(table) for w in words]
stripped

['1010']

In [172]:
words

['10/10']

In [151]:

for i in a.index:
    print(re.split(', | |-', a[i]))

['A', 'very', 'very', 'very', 'slow', 'moving', 'aimless', 'movie', 'about', 'a', 'distressed', 'drifting', 'young', 'man.', '', '']
['Not', 'sure', 'who', 'was', 'more', 'lost', '', '', 'the', 'flat', 'characters', 'or', 'the', 'audience', 'nearly', 'half', 'of', 'whom', 'walked', 'out.', '', '']
['Attempting', 'artiness', 'with', 'black', '&', 'white', 'and', 'clever', 'camera', 'angles', 'the', 'movie', 'disappointed', '', '', 'became', 'even', 'more', 'ridiculous', '', '', 'as', 'the', 'acting', 'was', 'poor', 'and', 'the', 'plot', 'and', 'lines', 'almost', 'non', 'existent.', '', '']
['Very', 'little', 'music', 'or', 'anything', 'to', 'speak', 'of.', '', '']
['The', 'best', 'scene', 'in', 'the', 'movie', 'was', 'when', 'Gerardo', 'is', 'trying', 'to', 'find', 'a', 'song', 'that', 'keeps', 'running', 'through', 'his', 'head.', '', '']
['The', 'rest', 'of', 'the', 'movie', 'lacks', 'art', 'charm', 'meaning...', 'If', "it's", 'about', 'emptiness', 'it', 'works', 'I', 'guess', 'becaus

In [152]:

count_word = Counter()
punct = ',.!?:;\'"-()*&^%$#@/\\'
for l in a:
    count_word.update(x.strip(punct).lower() for x in re.split(', | |-|:|\\n|\\t', l))

In [153]:
re.split(', | |-', a[0])[4].rstrip(punct).lower()

'slow'

In [154]:

re.split(', | |-|\\n|\\t', '\t1\nthe')

['', '1', 'the']

In [155]:
re.split(', | |-', 'slow-moving')

['slow', 'moving']

In [156]:
count_word

Counter({'a': 430,
         'very': 65,
         'slow': 6,
         'moving': 4,
         'aimless': 1,
         'movie': 180,
         'about': 50,
         'distressed': 1,
         'drifting': 1,
         'young': 4,
         'man': 13,
         '': 2131,
         'not': 73,
         'sure': 3,
         'who': 38,
         'was': 185,
         'more': 31,
         'lost': 4,
         'the': 845,
         'flat': 2,
         'characters': 35,
         'or': 40,
         'audience': 5,
         'nearly': 1,
         'half': 6,
         'of': 377,
         'whom': 2,
         'walked': 2,
         'out': 40,
         'attempting': 1,
         'artiness': 1,
         'with': 90,
         'black': 9,
         'white': 8,
         'and': 430,
         'clever': 5,
         'camera': 10,
         'angles': 2,
         'disappointed': 6,
         'became': 2,
         'even': 40,
         'ridiculous': 5,
         'as': 94,
         'acting': 43,
         'poor': 6,
         'plot': 28,
  

In [157]:
import operator
sorted_count = sorted(count_word.items(), key=operator.itemgetter(1), reverse=True)

In [158]:
sorted_count

[('', 2131),
 ('the', 845),
 ('a', 430),
 ('and', 430),
 ('of', 377),
 ('is', 339),
 ('this', 291),
 ('i', 267),
 ('it', 254),
 ('to', 253),
 ('in', 203),
 ('was', 185),
 ('movie', 180),
 ('film', 160),
 ('that', 156),
 ('for', 105),
 ('as', 94),
 ('but', 92),
 ('with', 90),
 ('one', 78),
 ('on', 76),
 ('you', 76),
 ('are', 75),
 ('not', 73),
 ('bad', 70),
 ("it's", 67),
 ('very', 65),
 ('so', 63),
 ('all', 63),
 ('just', 63),
 ('good', 58),
 ('at', 55),
 ('an', 54),
 ('be', 53),
 ('there', 51),
 ('about', 50),
 ('have', 49),
 ('by', 49),
 ('like', 48),
 ('from', 46),
 ('if', 45),
 ('acting', 43),
 ('time', 43),
 ('his', 41),
 ('really', 41),
 ('great', 41),
 ('or', 40),
 ('out', 40),
 ('even', 40),
 ('he', 39),
 ('who', 38),
 ('were', 38),
 ('has', 38),
 ('see', 36),
 ('characters', 35),
 ('my', 35),
 ('well', 34),
 ('most', 32),
 ('how', 32),
 ('more', 31),
 ('no', 31),
 ('only', 30),
 ('ever', 29),
 ('movies', 29),
 ('plot', 28),
 ('when', 28),
 ('story', 28),
 ('made', 27),
 ('some

# not needed

In [159]:
# create dictionary with frequency count
# use regex and Counter method
count_word1 = {}
punct = ',.!?\'"-*&^%$#@'
for i in a:
    for word in i:
        if word not in count_word:
            count_word[word] = 1
        else:
            count_word[word] += 1

In [160]:
imdb_neg

,desc,score,categ
0,"A very, very, very slow-moving, aimless movie ...",0,neg
1,Not sure who was more lost - the flat characte...,0,neg
2,Attempting artiness with black & white and cle...,0,neg
3,Very little music or anything to speak of.,0,neg
5,"The rest of the movie lacks art, charm, meanin...",0,neg
6,Wasted two hours.,0,neg
8,A bit predictable.,0,neg
15,It had some average acting from the main perso...,0,neg
25,This short film certainly pulls no punches.,0,neg
26,Graphics is far from the best part of the game.,0,neg


In [166]:
imdb_neg.desc[1]

'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  '

In [168]:
re.split(', | | ', imdb_neg.desc[1].lower())[:-2]

['not',
 'sure',
 'who',
 'was',
 'more',
 'lost',
 '-',
 'the',
 'flat',
 'characters',
 'or',
 'the',
 'audience',
 'nearly',
 'half',
 'of',
 'whom',
 'walked',
 'out.']

In [163]:
b = imdb_neg.desc.apply(lambda x: re.split(', | |-', x.lower())[:-2])
b

0      [a, very, very, very, slow, moving, aimless, m...
1      [not, sure, who, was, more, lost, , , the, fla...
2      [attempting, artiness, with, black, &, white, ...
3      [very, little, music, or, anything, to, speak,...
5      [the, rest, of, the, movie, lacks, art, charm,...
6                                  [wasted, two, hours.]
8                                 [a, bit, predictable.]
15     [it, had, some, average, acting, from, the, ma...
25     [this, short, film, certainly, pulls, no, punc...
26     [graphics, is, far, from, the, best, part, of,...
34                    [today, the, graphics, are, crap.]
39     [this, was, a, flick, doomed, from, its, conce...
40     [the, very, idea, of, it, was, lame, , , take,...
41             [i, wasn't, the, least, bit, interested.]
42     [not, only, did, it, only, confirm, that, the,...
43                        [but, it's, just, not, funny.]
44     [but, even, the, talented, carrell, can't, sav...
45     [his, co, stars, don't, 

In [164]:
# imdb_neg['pract'] = b

## decide what features to add, or what words to add to features

In [86]:
imdb['categ'] = imdb['score'].apply(lambda x:'pos' if x==1 else 'neg')

In [87]:
imdb_neg = imdb.copy()[imdb['score'] == 0]
imdb_pos = imdb.copy()[imdb['score'] == 1]


In [88]:
# imdb_pos['score'] = 'pos'
# imdb_neg['score'] = 'neg'

In [89]:
pos_words = imdb_pos['desc'].tolist()

In [90]:
neg_words = imdb_neg['desc'].tolist()

In [91]:
all_words = imdb['desc'].tolist()

In [92]:
def features(sentence):
    words = sentence.lower().split()
    return dict(('%s' % w, True) for w in words)

In [93]:
pos_featuresets = list(map(features, pos_words))

In [94]:
neg_featuresets = list(map(features, neg_words))